In [2]:
%pip install opencv-contrib-python
import cv2
import sys
import pickle
import glob
from time import time, sleep
import numpy as np
import json
from datetime import datetime
import os
from google.colab.patches import cv2_imshow

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'google.colab'

# Pré-definições

## Charuco e câmera

In [3]:
#Classe de erro
class Calibration_Error(Exception):pass

try:
    camera_id = int(sys.argv[1])
except:
    camera_id = 2#int(input("Camera id: "))
print('camera:', camera_id)



CHARUCOBOARD_ROWCOUNT = 3
CHARUCOBOARD_COLCOUNT = 4
squareLength = 0.190
markerLength = 0.148
image_h = 720
image_w = 1280
markerSize = 0.4 # Big aruco 

camera: 2


## Path

In [4]:
#data das fotos da calibração YYY-MM-DD
data = '2022-11-07' #datetime.today()
intrinsec_path = 'drive/MyDrive/Engenharia/Iniciação Científica DevOps/Github/camera-calibration/calibration_img/intrinsic/'#data_'+str(data)+'_camera'+str(camera_id)+'_'#'intrinsic\\data_'+str(data)+'_camera'+str(camera_id)+'_'
extrinsec_path = 'drive/MyDrive/Engenharia/Iniciação Científica DevOps/Github/camera-calibration/calibration_img/extrinsic/'#data_'+str(data)+'_camera'+str(camera_id)+'_'#'calibration_img\\extrinsic\\data_'+str(data)+'_camera'+str(camera_id)+'_'
output_path = 'drive/MyDrive/Engenharia/Iniciação Científica DevOps/Github/camera-calibration'

# Intrinsect

In [5]:
ARUCO_DICT = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
# Create constants to be passed into OpenCV and Aruco methods
CHARUCO_BOARD = cv2.aruco.CharucoBoard_create(
    squaresX=CHARUCOBOARD_COLCOUNT,
    squaresY=CHARUCOBOARD_ROWCOUNT,
    squareLength=squareLength,
    markerLength=markerLength,
    dictionary=ARUCO_DICT)


# Create the arrays and variables we'll use to store info like corners and IDs from images processed
corners_all = []  # Corners discovered in all images processed
ids_all = []  # Aruco ids corresponding to corners discovered
image_size = None  # Determined at runtime


for image_path in (os.listdir(intrinsec_path)):
    try:
        if "camera"+str(camera_id) in image_path:
            img = cv2.imread(intrinsec_path+image_path,0)
        else:
            continue
    except:
        continue
    try:
        corners, ids, _ = cv2.aruco.detectMarkers(
            image=img,
            dictionary=ARUCO_DICT)
    except:
        print(image_path,'error')
        continue
    # Outline the aruco markers found in our query image
    img = cv2.aruco.drawDetectedMarkers(
        image=img,
        corners=corners)

    # Get charuco corners and ids from detected aruco markers
    if ids is None:
        continue

    response, charuco_corners, charuco_ids = cv2.aruco.interpolateCornersCharuco(
        markerCorners=corners,
        markerIds=ids,
        image=img,
        board=CHARUCO_BOARD)

    # If a Charuco board was found, let's collect image/corner points
    # Requiring at least 20 squares

    #Must configure the threshold for response, i got 5 at LabTef in 2022-11-01
    print(response)
    if response > 5:
        # Add these corners and ids to our calibration arrays
        corners_all.append(charuco_corners)
        ids_all.append(charuco_ids)

        # # Draw the Charuco board we've detected to show our calibrator the board was properly detected
        img = cv2.aruco.drawDetectedCornersCharuco(
            image=img,
            charucoCorners=charuco_corners,
            charucoIds=charuco_ids)

        # # If our image size is unknown, set it now
        if not image_size:
            image_size = img.shape[::-1]

        #cv2_imshow(img)
        #cv2.waitKey(1)

    # sleep(1)

  # Destroy any open CV windows
cv2.destroyAllWindows()

print("num_images: {}".format(len(corners_all)))

# Make sure we were able to calibrate on at least one charucoboard by checking
# if we ever determined the image size
if not image_size:
    # Calibration failed because we didn't see any charucoboards of the PatternSize used
    print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
    # Exit for failure

# Now that we've seen all of our images, perform the camera calibration
# based on the set of points we've discovered
else:
    print("Calibrando...")
    calibration_error, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)

# print(cameraMatrix.reshape(-1).tolist())
# Print matrix and distortion coefficient to the console
print(f"cameraMatrix = {cameraMatrix}")
print(f"distCoeffs = {distCoeffs}")
print(f"calibration = {calibration_error}")




FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'drive/MyDrive/Engenharia/Iniciação Científica DevOps/Github/camera-calibration/calibration_img/intrinsic/'

# Extrinsic

In [ ]:
for image_path in (os.listdir(extrinsec_path)):

    try:
        if "camera"+str(camera_id) in image_path:
            img = cv2.imread(extrinsec_path+image_path,0)
        else:
            continue
    except:
        continue

    image_h, image_w = img.shape[:2]

    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    corners, ids, rejected = cv2.aruco.detectMarkers(img, ARUCO_DICT)
    rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(
        corners, markerSize, cameraMatrix, distCoeffs)

    cv2.aruco.drawDetectedMarkers(img, corners, ids)

    #7?
    marker_id = np.where(ids.reshape(-1) == 1)
    rvec = rvec[marker_id].reshape((1, -1))
    print(rvec.shape, ids, rvec.shape)
    rotation, _ = cv2.Rodrigues(rvec)
    translation = tvec[marker_id].reshape((3, 1))
    print("//", rotation.shape, translation.shape,
          np.array([[0, 0, 0, 1]]).shape)
    extrinsecs = cv2.hconcat([rotation, translation])
    print(extrinsecs.shape)
    last_colum = np.ones((1, 4))
    extrinsecs = cv2.vconcat([extrinsecs, last_colum])
    print(extrinsecs.shape, cameraMatrix.shape, distCoeffs.shape)
    break


(1, 0) [[0]] (1, 0)


error: ignored

# Salva resultados

In [ ]:
calibration_parameters = {
    "error": calibration_error,
    "resolution": {
        "height": image_h,
        "width": image_w,
    },
    "id": "{}".format(camera_id),
    "extrinsic": {
        "to": "{}".format(camera_id),
        "tf": {
          "shape": {
              "dims": [
                  {
                      "name": "rows",
                      "size": CHARUCOBOARD_ROWCOUNT
                  },
                  {
                      "name": "cols",
                      "size": CHARUCOBOARD_COLCOUNT
                  }
              ]
          },
            "type": "DOUBLE_TYPE",
            #"doubles": extrinsecs.reshape(-1).tolist()
        },
        "from": "1000"
    },
    "calibratedAt": data,
    "intrinsic": {
        "shape": {
			#Poderia alterar para rows e cols serem keys
            "dims": [
                {
                    "name": "rows",
                    "size": cameraMatrix.shape[0]
                },
                {
                    "name": "cols",
                    "size": cameraMatrix.shape[1]
                }
            ]
        },
        "type": "DOUBLE_TYPE",
        "doubles": cameraMatrix.reshape(-1).tolist()
    },
    "distortion": {
        "shape": {
            "dims": [
                {
                    "name": "rows",
                    "size": distCoeffs.shape[0]
                },
                {
                    "name": "cols",
                    "size": distCoeffs.shape[1]
                }
            ]
        },
        "type": "DOUBLE_TYPE",
        "doubles": distCoeffs.reshape(-1).tolist()
    }
}

with open(output_path+'/params_camera{}.json'.format(camera_id), 'w') as f:
    json.dump(calibration_parameters, f, indent=2)

print(json.dumps(calibration_parameters,indent=2))